In [ ]:
import tensorflow as tf
import os
from tflite_support import flatbuffers
from tflite_support import metadata as md
from tflite_support import metadata_schema_py_generated as md_fb

In [ ]:
savedModelPath = # Path to original tensorflow model
labelFilePath = # Path to label file
outputPath = # output filename; has to end with .tflite
imageHeight = # image height prescribed by the model
imageWidth = # image width prescribed by the model
normalizationMean = # mean for input normalization
normalizationStd = # standard deviation for input normalization

modelName = # model name
modelDescription = # description of the model
modelVersion = # model version
modelAuthor = # model author
modelLicense = # model license

In [ ]:
imported = tf.saved_model.load(savedModelPath)
print(imported.signatures)

In [ ]:
concrete_func = imported.signatures['default'] # use the default signature; signature names might differ from model to model
concrete_func.inputs[0].set_shape([None,imageWidth,imageHeight,3])
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.experimental_new_converter = True

In [ ]:
lite_model_content = converter.convert()
open(outputPath, "wb").write(lite_model_content)

In [ ]:
model_meta = md_fb.ModelMetadataT()
model_meta.name = modelName
model_meta.description = modelDescription
model_meta.version = modelVersion
model_meta.author = modelAuthor
model_meta.license = modelLicense

In [ ]:
input_meta = md_fb.TensorMetadataT()
output_meta = md_fb.TensorMetadataT()

input_meta.name="image"
input_meta.description = (
    "Input image to be classified. The expected image is {0} x {1}, with "
    "three channels (red, blue, and green) per pixel. Each value in the "
    "tensor is a single byte between 0 and 255.".format(imageWidth, imageHeight))
input_meta.content = md_fb.ContentT()
input_meta.content.contentProperties = md_fb.ImagePropertiesT()
input_meta.content.contentProperties.colorSpace = (md_fb.ColorSpaceType.RGB)
input_meta.content.contentPropertiesType = (md_fb.ContentProperties.ImageProperties)
input_normalization = md_fb.ProcessUnitT()
input_normalization.optionsType = (md_fb.ProcessUnitOptions.NormalizationOptions)
input_normalization.options = md_fb.NormalizationOptionsT()
input_normalization.options.mean = [normalizationMean]
input_normalization.options.std = [normalizationStd]
input_meta.processUnits = [input_normalization]
input_stats = md_fb.StatsT()
input_stats.max = [255]
input_stats.min = [0]
input_meta.stats = input_stats

# Creates output info.
output_meta = md_fb.TensorMetadataT()
output_meta.name = "probability"
output_meta.description = "Probabilities of the labels respectively."
output_meta.content = md_fb.ContentT()
output_meta.content.content_properties = md_fb.FeaturePropertiesT()
output_meta.content.contentPropertiesType = (md_fb.ContentProperties.FeatureProperties)
output_stats = md_fb.StatsT()
output_stats.max = [1.0]
output_stats.min = [0.0]
output_meta.stats = output_stats
label_file = md_fb.AssociatedFileT()
label_file.name = os.path.basename(labelFilePath)
label_file.description = "Labels for objects that the model can recognize."
label_file.type = md_fb.AssociatedFileType.TENSOR_AXIS_LABELS
output_meta.associatedFiles = [label_file]

In [ ]:
subgraph = md_fb.SubGraphMetadataT()
subgraph.inputTensorMetadata = [input_meta]
subgraph.outputTensorMetadata = [output_meta]
model_meta.subgraphMetadata = [subgraph]

b = flatbuffers.Builder(0)
b.Finish(model_meta.Pack(b),md.MetadataPopulator.METADATA_FILE_IDENTIFIER)
metadata_buf = b.Output()

In [ ]:
populator = md.MetadataPopulator.with_model_file(outputPath)
populator.load_metadata_buffer(metadata_buf)
populator.load_associated_files([labelFilePath])
populator.populate()